## Portfolio Optimizer

In [37]:
import random
import numpy as np
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cufflinks as cf
cf.go_offline()

# Portfolio consists of 10 stocks
# Define the list of stock tickers
tickers = ['AMZN', 'JPM', 'META', 'PG', 'GOOG', 'CAT', 'PFE', 'EXC', 'DE', 'JNJ']
# Download historical data for these stocks
data = yf.download(tickers, start='2014-01-01', end='2023-01-01')
close_price_df = data['Close']
# n = Number of stocks under consideration
n = len(close_price_df.columns) 
# Function to scale stock prices based on their initial starting price
def price_scaling(raw_prices_df):
    scaled_prices_df = raw_prices_df.copy()
    # Loop through each column in the df
    for i in raw_prices_df.columns:
        # Divide each price by the first price in the column
        scaled_prices_df[i] = raw_prices_df[i] / raw_prices_df[i].iloc[0] # Normalize the prices
    return scaled_prices_df

scaled_df = price_scaling(close_price_df) # Create df of scaled prices
scaled_df.head()

[*********************100%***********************]  10 of 10 completed


Ticker,AMZN,CAT,DE,EXC,GOOG,JNJ,JPM,META,PFE,PG
Date,,,,,,,,,,
2014-01-02 00:00:00+00:00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2014-01-03 00:00:00+00:00,0.996155,0.999444,1.004432,0.979757,0.992705,1.009008,1.007731,0.997258,1.001970,0.998882
2014-01-06 00:00:00+00:00,0.989095,0.986314,0.996122,0.986014,1.003773,1.014281,1.013572,1.045513,1.002955,1.001242
2014-01-07 00:00:00+00:00,1.000151,0.989540,1.000554,0.991167,1.023124,1.035812,1.001890,1.058673,1.009192,1.010926
2014-01-08 00:00:00+00:00,1.009925,0.991877,0.989807,0.988958,1.025253,1.034384,1.011338,1.064339,1.016087,0.996275


In [39]:
# Function generates random weights for the portfolio
def generate_portfolio_weights(n):
    weights = []
    for i in range(n):
        weights.append(random.random())
    # Ensures the sum of all weights add up to 1
    weights = weights/np.sum(weights)
    return weights

weights = generate_portfolio_weights(n)
print('Number of stocks under consideration = {}'.format(n))
print('Portfolio weights = {}'.format(np.round(weights, 2)))


Number of stocks under consideration = 10
Portfolio weights = [0.02 0.15 0.16 0.06 0.09 0.01 0.04 0.04 0.2  0.23]


#### Value of Portfolio = $ invested * (weights and stock values)

In [29]:
# Function returns a DataFrame with: 
#     (1) Daily value (position) of each individual stock over the specified time period
#     (2) Total daily value of the portfolio 
#     (3) Percentage daily return 

initial_investment = 1000000  # 1 million dollars


def asset_allocation(df, weights, initial_investment):
    portfolio_df = df.copy() # Create a copy of the DataFrame
    # Scale stock prices using the "price_scaling" function that we defined earlier (Make them all start at 1)
    scaled_df = price_scaling(df)
    #Assigns a weight to each stock in the portfolio
    for i, stock in enumerate(scaled_df.columns[0:]):
        portfolio_df[stock] = scaled_df[stock] * weights[i] * initial_investment
    # Sum up all values and place the result in a new column titled "portfolio value [$]" 
    portfolio_df['Portfolio Value [$]'] = portfolio_df[portfolio_df != 'Date'].sum(axis = 1, numeric_only = True)
            
    # Calculate the portfolio percentage daily return and replace NaNs with zeros
    portfolio_df['Portfolio Daily Return [%]'] = portfolio_df['Portfolio Value [$]'].pct_change(1) * 100 
    portfolio_df.replace(np.nan, 0, inplace = True)
    
    return portfolio_df

# Let's generate random weights 
print('Number of stocks under consideration = {}'.format(n))
weights = generate_portfolio_weights(n).round(6)
print('Portfolio weights = {}'.format(weights))

# Let's test out the "asset_allocation" function
portfolio_df = asset_allocation(close_price_df, weights, 1000000)
portfolio_df.head(3)

Number of stocks under consideration = 10
Portfolio weights = [0.051964 0.112016 0.136864 0.056604 0.137841 0.039817 0.110289 0.009054
 0.185649 0.159902]


Ticker,AMZN,CAT,DE,EXC,GOOG,JNJ,JPM,META,PFE,PG,Portfolio Value [$],Portfolio Daily Return [%]
Date,,,,,,,,,,,,
2014-01-02 00:00:00+00:00,51964.000000,112016.000000,136864.000000,56604.000000,137841.000000,39817.000000,110289.000000,9054.000000,185649.000000,159902.000000,1.000000e+06,0.000000
2014-01-03 00:00:00+00:00,51764.224000,111953.675068,137470.534540,55458.172589,136835.476194,40175.672177,111141.604963,9029.176763,186014.683180,159723.308490,9.995665e+05,-0.043347
2014-01-06 00:00:00+00:00,51397.312174,110482.896052,136333.286616,55812.339755,138361.101790,40385.628169,111785.794577,9466.072301,186197.530896,160100.534341,1.000322e+06,0.075630


In [73]:
# Define a function that performs interactive data visualization using Plotly Express
def plot_financial_data(df, title):

    fig = px.line(title = title)
    # For loop that plots all stock prices in the pandas dataframe df
    for i in df.columns[0:]:
        fig.add_scatter(x = df.index, y = df[i], name = i)
        fig.update_traces(line_width = 3)
        fig.update_layout({'plot_bgcolor': "white"})
    fig.show()

# Plot the portfolio percentage daily return
plot_financial_data(portfolio_df[['Portfolio Daily Return [%]']], 'Portfolio Percentage Daily Return [%]')

# Plot each stock position in our portfolio over time
# This graph shows how our initial investment in each individual stock grows over time
plot_financial_data(portfolio_df.drop(['Portfolio Value [$]', 'Portfolio Daily Return [%]'], axis = 1), 'Portfolio positions [$]')

# Plot the total daily value of the portfolio (sum of all positions)
plot_financial_data(portfolio_df[['Portfolio Value [$]']], 'Total Portfolio Value [$]')

$$ \text{Sharpe Ratio} = \frac{\text{Portfolio Return} - \text{Risk-Free Rate}}{\text{Portfolio Standard Deviation}} $$

In [82]:
# Simulation engine function: performs asset allocation, calculates portfolio statistical metrics including Sharpe ratio
# The function receives: portfolio weights and initial investment amount
# The function returns: Expected return, Expected volatility, Sharpe ratio, Return on investment, Final value in dollars

def simulation_engine(weights, initial_investment):
    # Perform asset allocation using the random weights (sent as arguments to the function)
    portfolio_df = asset_allocation(close_price_df, weights, initial_investment)
  
    # Return on investment is calculated using the last final value of the portfolio compared to its initial value
    return_on_investment = ((portfolio_df['Portfolio Value [$]'].iloc[-1:] - 
                             portfolio_df['Portfolio Value [$]'].iloc[0])/ 
                             portfolio_df['Portfolio Value [$]'].iloc[0]) * 100
  
    # Daily change of every stock in the portfolio (Note that we dropped the portfolio daily worth and daily % returns) 
    portfolio_daily_return_df = portfolio_df.drop(columns = ['Portfolio Value [$]', 'Portfolio Daily Return [%]'])
    portfolio_daily_return_df = portfolio_daily_return_df.pct_change(1) 
  
    # Portfolio Expected Return formula
    expected_portfolio_return = np.sum(weights * portfolio_daily_return_df.mean() ) * 252
  
    # Portfolio volatility (risk) formula
    # Risk of asset measured using standard deviation (cannot sum the risks of the individual assets)
    # Portfolio risk must consider correlations between assets within the portfolio which is indicated by the covariance 
    # The covariance determines the relationship between the movements of two random variables (moving together or inversely)
    covariance = portfolio_daily_return_df.cov() * 252 
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

    rf = 0.03 # Try to set the risk free rate of return to 1% (assumption)

    # Calculate Sharpe ratio
    sharpe_ratio = (expected_portfolio_return - rf)/expected_volatility 
    return expected_portfolio_return, expected_volatility, sharpe_ratio, portfolio_df['Portfolio Value [$]'].iloc[-1:].values[0], return_on_investment.values[0]
  

In [83]:
# Let's test out the "simulation_engine" function and print out statistical metrics
# Define the initial investment amount
initial_investment = 1000000
portfolio_metrics = simulation_engine(weights, initial_investment)

print('Expected Portfolio Annual Return = {:.2f}%'.format(portfolio_metrics[0] * 100))
print('Portfolio Standard Deviation (Volatility) = {:.2f}%'.format(portfolio_metrics[1] * 100))
print('Sharpe Ratio = {:.2f}'.format(portfolio_metrics[2]))
print('Portfolio Final Value = ${:.2f}'.format(portfolio_metrics[3]))
print('Return on Investment = {:.2f}%'.format(portfolio_metrics[4]))


Expected Portfolio Annual Return = 13.51%
Portfolio Standard Deviation (Volatility) = 17.63%
Sharpe Ratio = 0.60
Portfolio Final Value = $2543751.62
Return on Investment = 154.38%


#### Monte Carlo
runs trials with random inputs generated from an underlying distribution

In [86]:
# Set the number of simulation runs
sim_runs = 7
initial_investment = 1000000

# Placeholder to store values
weights_runs = np.zeros((sim_runs, n))
sharpe_ratio_runs = np.zeros(sim_runs)
expected_portfolio_returns_runs = np.zeros(sim_runs)
volatility_runs = np.zeros(sim_runs)
return_on_investment_runs = np.zeros(sim_runs)
final_value_runs = np.zeros(sim_runs)

for i in range(sim_runs):
    # Generate random weights 
    weights = generate_portfolio_weights(n)
    # Store the weights
    weights_runs[i,:] = weights
    # Call "simulation_engine" function and store Sharpe ratio, return and volatility
    # Note that asset allocation is performed using the "asset_allocation" function  
    expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i], final_value_runs[i], return_on_investment_runs[i] = simulation_engine(weights, initial_investment)
    print("Simulation Run = {}".format(i))   
    print("Weights = {}, Final Value = ${:.2f}, Sharpe Ratio = {:.2f}".format(weights_runs[i].round(3), final_value_runs[i], sharpe_ratio_runs[i]))   
    print('\n')


Simulation Run = 0
Weights = [0.101 0.099 0.061 0.069 0.069 0.096 0.018 0.175 0.144 0.167], Final Value = $2539623.73, Sharpe Ratio = 0.59


Simulation Run = 1
Weights = [0.125 0.09  0.123 0.138 0.079 0.128 0.08  0.01  0.128 0.1  ], Final Value = $2778770.44, Sharpe Ratio = 0.64


Simulation Run = 2
Weights = [0.15  0.065 0.066 0.086 0.1   0.102 0.168 0.16  0.093 0.011], Final Value = $2751811.41, Sharpe Ratio = 0.61


Simulation Run = 3
Weights = [0.148 0.067 0.206 0.026 0.069 0.068 0.163 0.008 0.195 0.05 ], Final Value = $3026689.33, Sharpe Ratio = 0.65


Simulation Run = 4
Weights = [0.131 0.071 0.145 0.109 0.037 0.148 0.074 0.055 0.148 0.083], Final Value = $2787319.57, Sharpe Ratio = 0.65


Simulation Run = 5
Weights = [0.229 0.162 0.054 0.145 0.053 0.151 0.052 0.057 0.049 0.049], Final Value = $2861707.05, Sharpe Ratio = 0.65


Simulation Run = 6
Weights = [0.251 0.056 0.041 0.132 0.126 0.188 0.104 0.066 0.016 0.021], Final Value = $2914932.77, Sharpe Ratio = 0.64




#### Markowitz Portfolio Optimization

In [ ]:
# List all Sharpe ratios generated from the simulation
sharpe_ratio_runs
# Return the index of the maximum Sharpe ratio (Best simulation run) 
sharpe_ratio_runs.argmax()
# Return the maximum Sharpe ratio value
sharpe_ratio_runs.max()
weights_runs
# Obtain the portfolio weights that correspond to the maximum Sharpe ratio (Golden set of weights!)
weights_runs[sharpe_ratio_runs.argmax(), :]
# Return Sharpe ratio, volatility corresponding to the best weights allocation (maximum Sharpe ratio)
optimal_portfolio_return, optimal_volatility, optimal_sharpe_ratio, highest_final_value, optimal_return_on_investment = simulation_engine(weights_runs[sharpe_ratio_runs.argmax(), :], initial_investment)
print('Best Portfolio Metrics Based on {} Monte Carlo Simulation Runs:'.format(sim_runs))
print('  - Portfolio Expected Annual Return = {:.02f}%'.format(optimal_portfolio_return * 100))
print('  - Portfolio Standard Deviation (Volatility) = {:.02f}%'.format(optimal_volatility * 100))
print('  - Sharpe Ratio = {:.02f}'.format(optimal_sharpe_ratio))
print('  - Final Value = ${:.02f}'.format(highest_final_value))
print('  - Return on Investment = {:.02f}%'.format(optimal_return_on_investment))

In [ ]:
# Create a DataFrame that contains volatility, return, and Sharpe ratio for all simualation runs
sim_out_df = pd.DataFrame({'Volatility': volatility_runs.tolist(), 'Portfolio_Return': expected_portfolio_returns_runs.tolist(), 'Sharpe_Ratio': sharpe_ratio_runs.tolist() })
sim_out_df

In [ ]:
# Plot volatility vs. return for all simulation runs
# Highlight the volatility and return that corresponds to the highest Sharpe ratio
import plotly.graph_objects as go
fig = px.scatter(sim_out_df, x = 'Volatility', y = 'Portfolio_Return', color = 'Sharpe_Ratio', size = 'Sharpe_Ratio', hover_data = ['Sharpe_Ratio'] )
fig.update_layout({'plot_bgcolor': "white"})
fig.show()


# Use this code if Sharpe ratio is negative
# fig = px.scatter(sim_out_df, x = 'Volatility', y = 'Portfolio_Return', color = 'Sharpe_Ratio', hover_data = ['Sharpe_Ratio'] )


In [ ]:
# Let's highlight the point with the highest Sharpe ratio
fig = px.scatter(sim_out_df, x = 'Volatility', y = 'Portfolio_Return', color = 'Sharpe_Ratio', size = 'Sharpe_Ratio', hover_data = ['Sharpe_Ratio'] )
fig.add_trace(go.Scatter(x = [optimal_volatility], y = [optimal_portfolio_return], mode = 'markers', name = 'Optimal Point', marker = dict(size=[40], color = 'red')))
fig.update_layout(coloraxis_colorbar = dict(y = 0.7, dtick = 5))
fig.update_layout({'plot_bgcolor': "white"})
fig.show()